# Libraries and Global Variables
to work with tabular data, it is recommended to install and use libraries such as pandas. In addition, we install and use pingouin to perform normality tests on the data to show how this dataset can be used.

### installing the libraries

In [9]:
!pip install pingouin
!pip install pandas
!pip install numpy

### variables and filename configuration

In [15]:
import pandas as pd
import pingouin as pg
import numpy as np
FILE_NAME = 'experiment_data.csv' # put the dataset csv file in the notebook's directory and check this name to match with the dataset's file name
ALPHA_LEVEL = 0.05 # for normality test
VIB_COLUMNS = ['vib_1', 'vib_2', 'vib_3', 'vib_4', 'vib_5'] # columns names containing the motors vibration intensities in each sample.
TEST_COLUMNS = ['magnitude', 'angle'] # columns names related to magnitude and angle selected by participants

# Loading Data and Grouping

In [19]:
df = pd.read_csv(FILE_NAME) # loading the file using pandas.read_csv
print(df.head()) # print the first 5 samples of the data to view the columns

   subject_id  session  data_point  magnitude     angle confidence  \
0           1        1           0   0.716670  1.968817     Medium   
1           1        1           1   0.516029  2.504551       High   
2           1        1           2   0.840066  1.148319       High   
3           1        1           3   0.969686  0.716135  Very High   
4           1        1           4   0.818451  1.995224       High   

   graph_time  interval_time  vib_1  vib_2  vib_3  vib_4  vib_5  
0   16.080935      20.018564      0     85     85      0      0  
1    2.591439       5.577325      0     85      0      0      0  
2    3.515985       6.109060      0      0    255    170      0  
3    2.861377      10.950164      0      0      0    255     85  
4    2.721418       5.772862      0     85    255      0      0  


### Aggregating the four vibration columns and create one pattern column to use it in the normality test

In [16]:
df['pattern'] = df[VIB_COLUMNS].astype(int).astype(str).agg('-'.join, axis=1)
print(f"Patterns Number: {df['pattern'].nunique()}\n") # printing the total number of patterns

Patterns Number: 51



# Performing Test
This test checks if each pattern's data comes from a normal distribution or not.

In [17]:
normality_results = []
grouped = df.groupby('pattern') # grouping the data based on each unique patterns

for pattern_value, group_data in grouped: # loop over unique patterns and their corresponding data
    test_data = group_data[TEST_COLUMNS].to_numpy()
    N = len(test_data)
    
    m_test = pg.multivariate_normality(test_data, alpha=ALPHA_LEVEL) # performing the test on the specific pattern data
    
    normality_results.append({
        'Pattern': pattern_value,
        'N': N,
        'HZ_Statistic': round(m_test.hz, 4), 
        'P_Value': round(m_test.pval, 4),
        'Is_Normal': m_test.normal,
        'Vib_Pattern': pattern_value.replace('-', ', ') 
    })

results_df = pd.DataFrame(normality_results)
results_df = results_df[['Vib_Pattern', 'N', 'HZ_Statistic', 'P_Value', 'Is_Normal']]
results_df = results_df.sort_values(by='N', ascending=False)

# Showing Results

In [18]:
print("\n--- Multivariate Normality Test Results (Henze-Zirkler) by Pattern ---")
print(f"Null Hypothesis ($H_0$): The data follow a multivariate normal distribution.")
print(f"A result of 'True' means $H_0$ is **not rejected** at the $\\alpha = {ALPHA_LEVEL}$ level.")
print("The 'Vib_Pattern' column order is: (vib_1, vib_2, vib_3, vib_4, vib_5)")
print("-" * 75)
print(results_df.to_markdown(index=False))
print("-" * 75)
print(f"\nSummary: {results_df['Is_Normal'].sum()} patterns out of {len(results_df)} total patterns did not reject the two-dimensional normality.")


--- 📊 Multivariate Normality Test Results (Henze-Zirkler) by Pattern ---
Null Hypothesis ($H_0$): The data follow a multivariate normal distribution.
A result of 'True' means $H_0$ is **not rejected** at the $\alpha = 0.05$ level.
The 'Vib_Pattern' column order is: (vib_1, vib_2, vib_3, vib_4, vib_5)
---------------------------------------------------------------------------
| Vib_Pattern       |   N |   HZ_Statistic |   P_Value | Is_Normal   |
|:------------------|----:|---------------:|----------:|:------------|
| 0, 0, 0, 0, 170   | 400 |        20.759  |    0      | False       |
| 0, 85, 85, 0, 0   | 400 |         2.5507 |    0      | False       |
| 0, 170, 170, 0, 0 | 400 |         6.9606 |    0      | False       |
| 0, 170, 255, 0, 0 | 400 |         8.1645 |    0      | False       |
| 0, 170, 85, 0, 0  | 400 |         1.5839 |    0.002  | False       |
| 0, 255, 0, 0, 0   | 400 |         5.3385 |    0      | False       |
| 0, 255, 170, 0, 0 | 400 |         5.4521 |    0    